In [1]:
using LinearAlgebra, SetRounding, Test

In [2]:
A = [1.0 2 3; 1 4 9; 1 1 1]

3×3 Matrix{Float64}:
 1.0  2.0  3.0
 1.0  4.0  9.0
 1.0  1.0  1.0

In [3]:
function qr(A)
    m,n = size(A)
    if n > m
        error("More columns than rows is not supported")
    end

    R = copy(A)
    Q = Matrix(1.0I, m, n)

    for j = 1:n-1 # first to last column
        y = R[j:end, j]
        y[1] += norm(y)
        w = y/norm(y)
        P = I - 2*w*w'

        R[j:end,j:end] = P * R[j:end, j:end]
        Q[:, j:end] = Q[:, j:end] * P
    end
    Q, R
end

Q, R = qr(A)
@test Q'Q ≈ I
@test Q*R ≈ A
@test R ≈ triu(R)

Test Passed

In [4]:
function rq(A)
    m,n = size(A)
    m == n || error("not square")
    ## SOLUTION
    R = copy(A)
    Q = Matrix(1.0I, n, n)
    for j = n:-1:2
        y = R[j, 1:j]            
        y[end] -= norm(y)
        w = y / norm(y)

        P = I - 2 * w * w'
        R[1:j, 1:j] = R[1:j, 1:j] * P
        Q[1:j, :] = P * Q[1:j, :]
    end
    R,Q
    ## END
end

R, Q = rq(A)
@test Q'Q ≈ I
@test R*Q ≈ A
@test R ≈ triu(R)

Test Passed

In [5]:
function ql(A)
    m,n = size(A)
    m == n || error("not square")
    ## TODO Create Q and L such that Q'Q == I and L is lower triangular
    ## SOLUTION
    L = copy(A)
    Q = Matrix(1.0I, n, n)
    for j = n:-1:2
        y = L[1:j, j]
        y[end] -= norm(y)
        w = y / norm(y)
        P = I - 2 * w * w'
        
        L[1:j, 1:j] = P * L[1:j, 1:j]
        Q[:,1:j] = Q[:,1:j] * P
    end
    Q,L
    ## END
end

Q,L = ql(A)
@test Q'Q ≈ I
@test Q*L ≈ A
@test L ≈ tril(L)

Test Passed

In [6]:
function lq(A)
    m,n = size(A)
    m == n || error("not square")
    ## TODO Create Q and L such that A = L*Q, Q'Q == I and L is lower triangular
    ## SOLUTION
    L = copy(A)
    Q = Matrix(1.0I, n, n)
    for k = 1:n-1
        y = L[k, k:end]
        y[1] -= norm(y)
        w = y / norm(y)
        P = I - 2 * w * w'

        L[k:end, k:end] = L[k:end, k:end] * P
        Q[k:end, :] = P * Q[k:end, :]
    end
    L,Q
    ## END
end

L, Q = lq(A)
@test Q'Q ≈ I
@test L*Q ≈ A
@test L ≈ tril(L)

Test Passed

In [7]:
A = [2 1 1 1; 1 2 1 1; 1 1 2 1; 1 1 1 2]

4×4 Matrix{Int64}:
 2  1  1  1
 1  2  1  1
 1  1  2  1
 1  1  1  2

In [8]:
function mycholesky(A)
    m,n = size(A)
    if n != m
        error("Matrix must be square")
    end
    if A != A'
        error("Matrix must be symmetric")
    end

    L = LowerTriangular(zeros(Float64, n, n))
    B = copy(A)

    for j = 1:n
        a = B[1,1]
        v = B[2:end, 1]
        if a <= 0
            error("Matrix is not symmetric positive definite")
        end
        L[j,j] = sqrt(a)
        L[j+1:end, j] = v / sqrt(a)

        B = B[2:end, 2:end] - v * v' / a
    end
    L
end

L = mycholesky(A)
@test L*L' ≈ A
@test L ≈ tril(L)

Test Passed

In [9]:
function mycholeskyu(A)
    m,n = size(A)
    if n != m
        error("Matrix must be square")
    end
    if A != A'
        error("Matrix must be symmetric")
    end

    U = UpperTriangular(zeros(Float64, n, n))
    B = copy(A)

    for j = n:-1:1
        a = B[end,end]
        v = B[1:end-1, end]
        if a <= 0
            error("Matrix is not symmetric positive definite")
        end
        U[j,j] = sqrt(a)
        U[1:j-1, j] = v / sqrt(a)

        B = B[1:end-1, 1:end-1] - v * v' / a
    end
    U
end

U = mycholeskyu(A)
@test U*U' ≈ A
@test U ≈ triu(U)

Test Passed